# 00 Stationarity and Unit Roots

ADF/KPSS, differencing, and spurious regression intuition.


## Table of Contents
- [Load macro series](#load-macro-series)
- [Transformations](#transformations)
- [ADF/KPSS tests](#adf-kpss-tests)
- [Spurious regression demo](#spurious-regression-demo)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Time-series econometrics notebooks build the classical toolkit you need before trusting macro regressions:
- stationarity + unit roots,
- cointegration + error correction,
- VAR dynamics and impulse responses.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Running tests without plotting or transforming the series first.
- Treating impulse responses as structural causality without an identification story.

## Matching Guide
- `docs/guides/08_time_series_econ/00_stationarity_unit_roots.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/08_time_series_econ/00_stationarity_unit_roots.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Learn the stationarity toolkit that prevents common macro mistakes:
- spurious regression
- over-trusting p-values on trending series
- misinterpreting dynamics



## Primer: pandas Time Series Essentials

### DatetimeIndex
Most time series work in pandas assumes your DataFrame is indexed by time.

```python
import pandas as pd

# idx = pd.to_datetime(df['date'])
# df = df.set_index(idx).sort_index()
# assert isinstance(df.index, pd.DatetimeIndex)
```

If you see weird behavior (resample errors, merges not aligning), check:
- `df.index.dtype`
- `df.index.min(), df.index.max()`
- `df.index.is_monotonic_increasing`

### Resampling
> **Goal:** convert one frequency to another.

Common examples:
- daily -> month-end
- monthly -> quarter-end

```python
# month-end last value
# df_me_last = df.resample('ME').last()

# month-end mean
# df_me_mean = df.resample('ME').mean()

# quarter-end mean
# df_q_mean = df.resample('QE').mean()
```

**Interpretation matters.** For economic series:
- using `.last()` treats the end-of-period value as “the period’s value”
- using `.mean()` treats the period average as “the period’s value”

Neither is universally correct; you should choose based on measurement and use case.

### Alignment and merging
When joining multiple time series, you need to ensure they share:
- the same index type (`DatetimeIndex`)
- the same frequency convention (month-end vs month-start; quarter-end vs quarter-start)

```python
# Example: join two series and inspect missingness
# df = df1.join(df2, how='outer').sort_index()
# print(df.isna().sum())
```

### Lags and rolling windows
> **Lag:** use past values as features.

```python
# lag 1 period
# df['x_lag1'] = df['x'].shift(1)

# rolling mean (past-only)
# df['x_roll12'] = df['x'].rolling(12).mean()
```

### Common gotchas
- `shift(-1)` uses the future (leakage for forecasting).
- `rolling(..., center=True)` uses future values.
- Always `dropna()` after creating lags/rolls to get clean modeling rows.

One more gotcha:
- If you resample daily -> monthly and then create lags, your lag is “one month” (not one day). Lags are measured in the current index frequency.


## Primer: Classical time-series econometrics with statsmodels (ADF/KPSS, VAR)

This repo already uses time-aware evaluation for ML.
This primer introduces the “classical” time-series econometrics toolkit in `statsmodels`.

### Stationarity and unit roots (ADF / KPSS)
Two common tests:
- **ADF**: null = unit root (nonstationary)
- **KPSS**: null = stationary

```python
from statsmodels.tsa.stattools import adfuller, kpss

# x is a 1D array-like (no missing)
# adf_stat, adf_p, *_ = adfuller(x)
# kpss_stat, kpss_p, *_ = kpss(x, regression='c', nlags='auto')
```

Interpretation habit:
- If ADF p-value is small → evidence against unit root.
- If KPSS p-value is small → evidence against stationarity.

### VAR: multivariate autoregression
VAR models multiple series together:
```python
from statsmodels.tsa.api import VAR

# df: DataFrame of stationary-ish series with a DatetimeIndex
# model = VAR(df)
# res = model.fit(maxlags=8, ic='aic')  # or choose lags manually
# print(res.summary())
```

Useful tools:
```python
# res.test_causality('y', ['x1', 'x2'])      # Granger causality tests
# irf = res.irf(12)                         # impulse responses to 12 steps
# irf.plot(orth=True)                       # orthogonalized (ordering matters)
```

### Practical cautions
- Nonstationary series can create **spurious regression** results.
- IRFs depend on identification choices (e.g., Cholesky ordering).
- Macro series are revised and can have structural breaks; treat results as conditional and fragile.


<a id="load-macro-series"></a>
## Load macro series

### Goal
Load the macro monthly panel.



### Your Turn: Load panel_monthly.csv (or sample)


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'panel_monthly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

df = df.dropna().copy()
df.head()



<a id="transformations"></a>
## Transformations

### Goal
Create stationary-ish transformations (diff, pct change, log diff).



### Your Turn: Differences and growth rates


In [ ]:
import numpy as np

# TODO: Pick a few series and create transformations
tmp = df[['CPIAUCSL', 'UNRATE', 'INDPRO']].astype(float).copy()
tmp['dCPI'] = tmp['CPIAUCSL'].diff()
tmp['dUNRATE'] = tmp['UNRATE'].diff()

# log-diff for industrial production (example)
x = tmp['INDPRO'].where(tmp['INDPRO'] > 0)
tmp['dlog_INDPRO'] = np.log(x).diff()

tmp = tmp.dropna()
tmp.head()



<a id="adf-kpss-tests"></a>
## ADF/KPSS tests

### Goal
Run stationarity diagnostics on levels vs transformed series.



### Your Turn: ADF and KPSS


In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# TODO: Choose one series and compare levels vs diff
x = df['CPIAUCSL'].astype(float).dropna()
dx = x.diff().dropna()

out = {
    'adf_p_level': adfuller(x)[1],
    'adf_p_diff': adfuller(dx)[1],
    'kpss_p_level': kpss(x, regression='c', nlags='auto')[1],
    'kpss_p_diff': kpss(dx, regression='c', nlags='auto')[1],
}
out



<a id="spurious-regression-demo"></a>
## Spurious regression demo

### Goal
Show how levels-on-levels regressions can look good for the wrong reasons.



### Your Turn: Levels vs differences


In [ ]:
import statsmodels.api as sm

tmp2 = df[['CPIAUCSL', 'INDPRO']].astype(float).dropna()

# Levels regression
res_lvl = sm.OLS(tmp2['CPIAUCSL'], sm.add_constant(tmp2[['INDPRO']], has_constant='add')).fit()

# Differences regression
d = tmp2.diff().dropna()
res_diff = sm.OLS(d['CPIAUCSL'], sm.add_constant(d[['INDPRO']], has_constant='add')).fit()

print('R2 levels:', res_lvl.rsquared)
print('R2 diffs :', res_diff.rsquared)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# TODO: Validate your time series table is well-formed.
# Example (adjust variable names):
# assert isinstance(df.index, pd.DatetimeIndex)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 30
#
# TODO: If you built transformed series (diff/logdiff), confirm no future leakage.
# Hint: transformations should only use past/current values (shift/diff), never future.
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Load macro series</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — Load macro series
import pandas as pd

path = PROCESSED_DIR / 'panel_monthly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

df = df.dropna().copy()
df.head()
```

</details>

<details><summary>Solution: Transformations</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — Transformations
# Example: difference CPI and unemployment
df_t = df[['CPIAUCSL', 'UNRATE']].astype(float).copy()
df_t['dCPI'] = df_t['CPIAUCSL'].diff()
df_t['dUNRATE'] = df_t['UNRATE'].diff()
df_t = df_t.dropna()
df_t.head()
```

</details>

<details><summary>Solution: ADF/KPSS tests</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — ADF/KPSS tests
from statsmodels.tsa.stattools import adfuller, kpss

x = df['CPIAUCSL'].astype(float).dropna()
dx = x.diff().dropna()

adf_p_level = adfuller(x)[1]
adf_p_diff = adfuller(dx)[1]
kpss_p_level = kpss(x, regression='c', nlags='auto')[1]
kpss_p_diff = kpss(dx, regression='c', nlags='auto')[1]

{'adf_p_level': adf_p_level, 'adf_p_diff': adf_p_diff, 'kpss_p_level': kpss_p_level, 'kpss_p_diff': kpss_p_diff}
```

</details>

<details><summary>Solution: Spurious regression demo</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 00_stationarity_unit_roots — Spurious regression demo
import statsmodels.api as sm

# Levels-on-levels can look 'significant' even when dynamics are mis-specified.
tmp = df[['CPIAUCSL', 'INDPRO']].astype(float).dropna()
res_lvl = sm.OLS(tmp['CPIAUCSL'], sm.add_constant(tmp[['INDPRO']], has_constant='add')).fit()
res_diff = sm.OLS(tmp['CPIAUCSL'].diff().dropna(), sm.add_constant(tmp['INDPRO'].diff().dropna(), has_constant='add')).fit()

(res_lvl.rsquared, res_diff.rsquared)
```

</details>

